In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import torch
import os
import sys
import random

module_path = os.path.abspath(os.path.join('..'))
abs_path = "/content/drive/MyDrive/atml"
sys.path.append(abs_path+"/models")
sys.path.append(abs_path+"/train")
sys.path.append(abs_path+"/datasets")

from datasets import load_dsprites, CustomDSpritesDataset
from beta_vae import BetaVAEDSprites
from control_vae import ControlVAEDSprites
from factor_vae import FactorVAEDSprites
from entanglement_metric import compute_mig

# Fix seed 
torch.manual_seed(2)
random.seed(2)
np.random.seed(2)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
device

device(type='cuda')

In [ ]:
def load_vae_model(model_type, model_path, *args, **kwargs):
    model = None

    if model_type == "beta_vae":
        model = BetaVAEDSprites(*args, **kwargs)
        checkpoint = torch.load(abs_path + model_path)
        model.load_state_dict(checkpoint['state_dict'])
    elif model_type == "control_vae":
        # model = ControlVAEDSprites(*args, **kwargs)
        model = torch.load(abs_path + model_path)
    elif model_type == "factor_vae":
        # model = FactorVAEDSprites(*args, **kwargs)
        model = torch.load(abs_path + model_path)
    else:
      raise NotImplementedError()

    model.eval()

    return model

In [ ]:
dataset = CustomDSpritesDataset(load_dsprites(abs_path + "/datasets/dsprites.npz", False))

In [ ]:
for beta in [1, 4]:
  model = load_vae_model('beta_vae', f'/experiments/trained_models/betavae_models/betavae_beta{beta}_e50_alldata_n.pth.tar')
  print(f"beta: {beta} | mig: {compute_mig(model, dataset, device=device):.3f}")

beta: 1 | mig: 0.053
beta: 4 | mig: 0.219


In [ ]:
beta_controller_args = {
    'C' : 0.5,
    'C_max' : 12,
    'C_step_val' : 0.15,
    'C_step_period' : 5000,
    'Kp' : 0.01,
    'Ki' : -0.001,
    'Kd' : 0.0
}

for cmax in [8, 10, 12]:
  model = load_vae_model('control_vae', f'/experiments/trained_models/controlvae_epoch50_lr1e2_Cmax{cmax}.dat', beta_controller_args)
  print(f"cmax: {cmax} | mig: {compute_mig(model, dataset, device=device):.3f}")

cmax: 8 | mig: 0.267
cmax: 10 | mig: 0.294
cmax: 12 | mig: 0.293


In [ ]:
for gamma, lrd in [(5, '0.0001'), (40, '5e-05')]:
  model = load_vae_model('factor_vae', f'/experiments/trained_models/factorvae_epochs50_gamma{gamma}_lrvae0.01_lrd{lrd}.dat', beta_controller_args)
  print(f"gamma: {gamma} | mig: {compute_mig(model, dataset, device=device):.3f}")

gamma: 5 | mig: 0.158
gamma: 40 | mig: 0.119
